In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
plt.style.use('seaborn-notebook')
%matplotlib notebook

In [ ]:
name = 'Esc3'
df = gpd.read_file('../data/interim/shapes/AP_Geotermica_' + name + '.shp')
df = df[df.geometry.notnull()].to_crs({'init': 'epsg:4326'}) 
print (len(df))

In [ ]:
states = gpd.read_file('../data/interim/shapes/mex-states.shp')
# Reproject the file
states = states.to_crs({'init': 'epsg:4326'}) 

In [ ]:
import json, os
with open(os.path.join('../data/interim/', 'trans-regions.json'), 'r') as fp:
    trans_regions = json.load(fp)

In [ ]:
loadzones = gpd.read_file('../data/interim/shapes/Mask_T.shp')
loadzones = loadzones.to_crs({'init': 'epsg:4326'}) 
loadzones['trans-region'] = loadzones['ID'].astype(int).map('{0:02}'.format).map(trans_regions)

In [ ]:
loadzones.crs

In [ ]:
#loadzones.to_file("test.geojson", driver='GeoJSON')

In [ ]:
base = states.plot(color='white', edgecolor='black')
join = gpd.sjoin(df, loadzones, op='within')
#join = gpd.overlay(loadzones, df, how='intersection')
loadzones.plot(ax=base,color='white', edgecolor='black', label='Loadzones')
df.plot(color='red', ax=base, label='Outside', markersize=5)
#join.plot(ax=base, color='green', legend=True)
join.plot(ax=base, color='green', label='Inside', markersize=5)
plt.title(name)
plt.xlabel('lng')
plt.ylabel('lat')
plt.legend();
#plt.savefig(name + '.png', dpi=300)

In [ ]:
from itertools import zip_longest

#join = gpd.sjoin(df, loadzones, op='within')
load_zones_out = loadzones[~loadzones['trans-region']
                          .isin(join['trans-region'].unique())]['trans-region']
load_zones_in = join.groupby('trans-region')['geometry'].count().index.values
#join = gpd.overlay(loadzones, df, how='intersection')
with open(name + '.txt', 'w') as fpd:
    fpd.write('Trans regions included:\t{0}\n'.format(len(join['trans-region'].unique())))
    fpd.write('Trans regions not included: \t{0}\n'
              .format(len(loadzones[~loadzones['trans-region']
                                    .isin(join['trans-region'].unique())])))
    fpd.write('Total points:\t{0}\n'.format(len(df)))
    fpd.write('Points inside:\t{0}\n'.format(len(join)))
    fpd.write('Points outside:\t{0}\n'.format(len(df) - len(join)))
    fpd.write('Trans_in\tTrans_out\n')
    for values in zip_longest(*[load_zones_in, load_zones_out]):
        fpd.write('{0}\t{1}\n'.format(values[0],values[1]))

In [ ]:
no_1 = gpd.sjoin(df, loadzones, op='within')

In [ ]:
grouped = no_1.groupby(['trans-region'])['ID'].count()

In [ ]:
grouped